# PLANNING

This notebook describes the planning.py module, which covers Classical Planning from Chapter 10 & Planning & Acting in Real World Chapter 11.

Let us first import the requires modules.

In [1]:
from planning import *
from utils import expr

## **The Planning Problem** 
> To find an executable sequence of actions that achieves a given goal when performed starting in a given state.

Motivation of the planning process is to reason about possible course of actions that will change the environment in order to reach the goal (task).

## Planning v/s Scheduling

We need to understand the difference between the terms:




**Scheduling** - assigns in time resources to separate processes.

Given: _resources, actions and constraints._

       Form an appropriate schedule that meets the constraints

       Arrange the actions, assign resources and satisfy the constrains.


**Planning** - considers possible interaction among components of plan.

Given: _the initial state, goal state, operators._

       Find a sequence of operators that will reach the goal state from the initial state

       Select appropriate actions, arrange the actions and consider the causalities

## Assumptions in Classical Planning

Classical planners make at least the following assumptions:

- the agent's actions are deterministic; that is, the agent can predict the consequences of its actions.
- there are no external events beyond the control of the agent that change the state of the world.
- the world is fully observable; thus, the agent can observe the current state of the world.
- time progresses discretely from one state to the next.
- goals are predicates of states that must be achieved or maintained.


## Representations For Planning:

1. **STRIPS** 
The simplest language used for formalizing actions is the STRIPS language. In STRIPS, the state variables have the domain {0,1} (equivalently {FALSE, TRUE}), and an action consists of three sets of state variables, the PRECONDITION, the ADD={a1,a2,...,an} list, and the DELETE={d1,d2,...,dm} list (it is assumed that ADD and DELETE don't intersect.) 

2. **ADL**
Action description language (ADL) is an automated planning and scheduling system in particular for robots. It is considered an advancement of STRIPS. We won't go ino its details here.

3. **PDDL**
("Planning Domain Definition Language") is a recent atempt to standardize planning domain and problem description languages. It was developed mainly to make the 1998/2000 International Planning Competitions possible. 

The differences of PDDL in comparison to STRIPS are:

  - The PRECONDITION may be an arbitrary Boolean combination of atomic facts about the state variables. Atomic facts say something about one state variable, for example a=0 or b=1. A precondition could for example be (a=1)∨¬(b=1∧c=0).
  - Instead of the unconditional assignments represented by ADD and DELETE, the effects may be conditional. This means that the effects are of the form, IF condition THEN a := v where the condition is a Boolean combination of facts. STRIPS corresponds to PDDL with trivial conditions that are always true (the condition is the constant TRUE).
  - Goals may be Boolean combinations of atomic facts (formulas). 

We will be using **PDDL** consistent with the book.

#PDDL 
The PDDL (Planning Domain Definition Language)   allows us to express all actions with one action schema.

It can be used to define a problem:
1. initial state
2. Actions that are available in a state
3. Result of applying an action
4. The goal test


The `PDDL class` includes:

1. **`init(self, initial_state, actions, goal_test)`**: the constructor creates a knowledge base with initial state, initialises actions and `goal_test_func` function with `goal_test`.
2. **`goal_test(self)`**: initialises `goal_test` with `kb`.
3. **`act(self, action)`**:  performs the action given as argument, along with checks preformed on pre-conditions.


PDDL is a domain definition language. It is used to define the properties of a domain, the predicates which are used and the action definition.  A predicate defines the property of an object which can be true or false, e.g. yellow t-shirt. Yellow is the property and t-shirt is the object. 

#ACTIONS
Actions are described by a set of action schemas that implicitly define the `class Actions`.
The schema consists of a action name, a list of variables used in the schema, a precondition (positive and negative) and effect (positive and negative). 

The  **precondition** of an action specifies when the action can be carried out. The precondition of an action is a proposition that must be true before the action can be carried out. In terms of constraints, the robot is constrained to only be able to choose an action for which the precondition is true. 
For example, in the block problem move(x,y) has preconditions clear(x) and clear(y). The action move(x,table) is always possible.

The **effect** of an action specifies the resulting state.
Each problem described in chapter 10 has:
- an initial state
- a goal
- actions with preconditions and effects (these action are split into _pos (positive conditions) and _neg (negative conditions))
Each problem thus requires a solution which is satisfied by some initial conditions and solved using actions which meet the preconditions and effects.
 
We say that action `a` is **applicable** in state s if the preconditions are satisfied by s.

Now that we have a sense of the representations let's go through some examples.

## Factored Representation:

In contrast to atomic representation where state is a single indivisible entity (figure on the left), in factored representation state is a collection of variables (figure on the right below).

<img src="files/images/factored_state_representation.png"> The problems we see ahead are in factored state representation.
Now that we have a sense of the representations let's see some examples.

# AIR CARGO PROBLEM

The problem is to transport cargo via plane to & from airports.

###Real Life Cargo Transportaion by Planes instance:
- 10 airports
- 50 aircrafts
- 200 pieces of cargo
- no. of states 10<sup>50</sup> x (50 + 10)<sup>200  $\approx$ 10<sup>405</sup>
- minimum number of actions 50 x 9 =450 (all cargo located on airport with no planes)
- maximum number of actions 50<sup>200</sup> x 9 $\approx$ 10<sup>340</sup> (all cargo and aircrafts in one airport)

Practically: The number of particles in the universe is about **_10<sup>87</sup>_**.

That is where automated planning research comes in to solve such problems using classical planning (or other algorithms).

#####For now we will stick to the example from the book from Fig. 10.1:


To define a problem we have to define an initial state (predicates which are true at the beginning of the problem) and a goal state (predicates which are true at the end of the problem). 


The following code defines the initial state for the problem


In [2]:
def air_cargo():
    init = [expr('At(C1, SFO)'),     
            expr('At(C2, JFK)'),
            expr('At(P1, SFO)'),
            expr('At(P2, JFK)'),
            expr('Cargo(C1)'),
            expr('Cargo(C2)'),
            expr('Plane(P1)'),
            expr('Plane(P2)'),
            expr('Airport(JFK)'),
            expr('Airport(SFO)')]

`init` defines the initial state of the problem.

**`expr('At(C1, SF0)')`** represents the predicate **Cargo 1 at San Fransisco**

**`expr('At(P1, SF0)')`** represents the predicate **Plane 1 at San Fransisco**    ... and so on.



The following code defines the goal_test() function which tests if the solution achieves goal or not. 'required' states the goal:

In [3]:
def goal_test(kb):
        required = [expr('At(C1 , JFK)'), expr('At(C2 ,SFO)')]
        for q in required:
            if kb.ask(q) is False:
                return False
        return True

For this problem the Goal is   **`(At(C1 , JFK ) ∧ At(C2 , SFO))`** i.e., **Cargo 1 ar JFK and Cargo 2 at San Fransisco** which is written as:

`required = [expr('At(C1 , JFK)'), expr('At(C2 ,SFO)')]`

The function `goal_test(kb)` takes a knowledge base as argument, and for every predicate in `required` (goal), it checks the `ask` function from `KB class`. The `ask` function returns value `True` or `False` accordingly if predicate in `required` meets preconditions or not (defined ahead).


Actions such as 'load', 'unload'  and fly are defined with preconditions and effects accompanying them.

In [4]:
# Actions
#  Load
precond_pos = [expr("At(c, a)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]
precond_neg = []
effect_add = [expr("In(c, p)")]
effect_rem = [expr("At(c, a)")]
load = Action(expr("Load(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Unload
precond_pos = [expr("In(c, p)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]
precond_neg = []
effect_add = [expr("At(c, a)")]
effect_rem = [expr("In(c, p)")]
unload = Action(expr("Unload(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])

#  Fly
#  Used 'f' instead of 'from' because 'from' is a python keyword and expr uses eval() function
precond_pos = [expr("At(p, f)"), expr("Plane(p)"), expr("Airport(f)"), expr("Airport(to)")]
precond_neg = []
effect_add = [expr("At(p, to)")]
effect_rem = [expr("At(p, f)")]
fly = Action(expr("Fly(p, f, to)"), [precond_pos, precond_neg], [effect_add, effect_rem])

`precond_` is used to denote predicates which must be True **before** the action.
eg: 

`effect_` is used to denote predicates which must be True **after** the action.

`precon_pos` & `effect_add` are used to denote predicates which must be **True**.

`precon_neg` & `effect_rem` are used to denote predicates which must be **False**.


Eg: in  Action `load`

`precond_pos = [expr("At(c, a)"), expr("At(p, a)"), expr("Cargo(c)"), expr("Plane(p)"), expr("Airport(a)")]` is a  precondition which must be **True**   denoting predicate _Cargo c at Airport a & Plane p at Airport a_.

`precond_neg = []` is a precondition which must be **False**. 			

`effect_add = [expr("In(c, p)")]` is an effect which must be **True**  denoting the predicate _Cargo c in Plane p_.

`effect_rem = [expr("At(c, a)")]`is an effect which must be **False** denoting the predicate _Cargo c at Airport a_.

`load = Action(expr("Load(c, p, a)"), [precond_pos, precond_neg], [effect_add, effect_rem])`   thus define the action **`Load(c, p, a)** with the preconditions it must follow and the effects it leads to.



Finally, the function returns the defined problem using PDLL.

`return(PDLL(init, [load, unload, fly], goal_test)`

 which defines the problem as a whole with its initial state (`init`), the actions possible(` [load, unload, fly]`) and the goal(`goal_test`).

A solution to the air_cargo problem is as follows:

In [2]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")]

where `expr("Load(C1 , P1, SFO)")` means _load the Cargo C1 in Plane P1 at San Fransico_. In order for this action to take place the preconditions we specified while defining the action `Load` must be met.  The effects from this action are then carried forward i.e., they now are an existing state and it should be sought that these effects do **not** clash with preconditions of actions ahead, otherwise the action can't be completed. 

Following lines describe the _plan_ of the above solution:

`expr("Load(C1 , P1, SFO)") ` -->  _Load Cargo C1 in Plane P1 at San Fransico_

`expr("Fly(P1, SFO, JFK)")`    -->  _Fly Plane P1 from San Fransisco to JFK_

`expr("Unload(C1, P1, JFK)")` -->  _Unload Cargo C1 from Plane P1 at JFK_

`expr("Load(C2, P2, JFK)")`   -->  _Load Cargo C2 in Plane P2 at JFK_

`expr("Fly(P2, JFK, SFO)")`  -->  _Fly Plane P2 from JFK to San Fransisco_

`expr("Unload (C2, P2, SFO)")`   -->  _Unload Cargo C2 from Plane P2 at San Fransisco_


We then execute the action on the state's kb.

In [3]:
a = air_cargo()

for action in solution:
    a.act(action)

where each action is acted upon leading to a state which is tested below to be the goal state or not.

In [4]:
a.goal_test()

True

Yes, the solution is correct. You may try any other solution to check if it achieves the goal or not.

## Another related example - The Dock-Worker Robots (DWR) Domain

It is specifically an example of automated planning, with the following scenario:
- harbour with several locations (docs)
- docked ships
- storage area for containers
- parking area for trains, trucks

- Goal: 
   	1. cranes to load and unload ships
   	2. robot carts to move around containers

Like the air cargo transport problem, the actions can be sought to be  **put, take, load, unload, move**.

The following figure depicts a simple instance of the problem where the initial state is S<sub>0</sub> (robot at location 2 and container on pallet in location 1) and the goal state is S<sub>5</sub> (container on robot at location 2).  <img src="files/images/dwr_state_transition.png">   The state transitions are accompanied by the following actions:

- S<sub>0</sub> (robot at location 2 and container on pallet in location 1) -- **initial state**; `At(robot, location2), At(container, location1), At(container, pallet)`
- S<sub>1</sub> (robot at location 2 and container on crane in location 1) -- action `take(crane, container)` 
- S<sub>2</sub> (robot at location 1 and container on pallet in location 1) -- action `move(location2, location1), put(container, pallet)`
- S<sub>3</sub> (robot at location 1 and container on crane in location 1) -- action `take(crane, container)`
- S<sub>4</sub> (robot at location 1 and container on robot in location 1) -- action `load(robot, container)`
- S<sub>5</sub> (robot at location 2 and container on robot in location 2) -- action  `move(location1, location2)` ; **goal state**


# SPARE TIRE PROBLEM from Figure 10.2:

The Spare Tire problem is to replace a flat tire on axle with a spare one. 

The problem follows the same structure of function as before, only the predicates are changed.


In [5]:
init = [expr('Tire(Flat)'),
            expr('Tire(Spare)'),
            expr('At(Flat, Axle)'),
            expr('At(Spare, Trunk)')]


required = [expr('At(Spare, Axle)'), expr('At(Flat, Ground)')]

`init ` states the initial state:

`expr('Tire(Flat)')` --> _Flat Tire_

`expr('Tire(Spare)')` --> _Spare Tire_

`expr('At(Flat, Axle)')` --> _Flat Tire on Axle_

`expr('At(Spare, Trunk)')` --> _Spare Tire in Trunk_


`required = [expr('At(Spare, Axle)'), expr('At(Flat, Ground)')]` states the goal i.e., _Spare tire must be on Axle, and Flat tire on Ground_

Just a sidenote, the action `At()` means same as _on, in, at_ in the problem reference.

In [6]:
# Actions

# Remove
precond_pos = [expr("At(obj, loc)")]
precond_neg = []
effect_add = [expr("At(obj, Ground)")]
effect_rem = [expr("At(obj, loc)")]
remove = Action(expr("Remove(obj, loc)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# PutOn
precond_pos = [expr("Tire(t)"), expr("At(t, Ground)")]
precond_neg = [expr("At(Flat, Axle)")]
effect_add = [expr("At(t, Axle)")]
effect_rem = [expr("At(t, Ground)")]
put_on = Action(expr("PutOn(t, Axle)"), [precond_pos, precond_neg], [effect_add, effect_rem])

# LeaveOvernight
precond_pos = []
precond_neg = []
effect_add = []
effect_rem = [expr("At(Spare, Ground)"), expr("At(Spare, Axle)"), expr("At(Spare, Trunk)"),
              expr("At(Flat, Ground)"), expr("At(Flat, Axle)"), expr("At(Flat, Trunk)")]
leave_overnight = Action(expr("LeaveOvernight"), [precond_pos, precond_neg],
                         [effect_add, effect_rem])

The problem involves 3 actions:

1. `Remove(obj, loc)` --> Remove object _obj_ from location _loc_.
2. `PutOn(t, Axle)` --> Put tire_t_ on _Axle_
3. `Leaveovernight` --> Leave tire overnight  _(we assume the neighbourhood is good and we find the tire at the same place the next day)_

solution to the spare tire problem from the book is as follows:

In [7]:
solution = [expr("Remove(Flat, Axle)"),
                expr("Remove(Spare, Trunk)"),
                expr("PutOn(Spare, Axle)")]

Following lines describe the _plan_ of the above solution:

`expr("Remove(Flat, Axle)")` --> _Remove Flat tire from Axle_

`expr("Remove(Spare, Trunk)")` --> _Remove Spare tire from Trunk_

`expr("PutOn(Spare, Axle)")` --> _Put Spare tire on Axle_


let's test it in the same way as before :

In [8]:
s = spare_tire()

for action in solution:
    s.act(action)

s.goal_test()

True

The solution achieves the goal.

# THE BLOCKS WORLD from Figure 10.3 :

 The above image below shows the transitions between different states which are possible for 3 blocks. Watch how the rightmost transition is what we follow in the book.   <img src="files/images/blocks_world.png">